In [95]:
#Webscrape https://www.rebellegirls.com/yarnfiber.htm for list of offered brands
import requests
from bs4 import BeautifulSoup
import pandas as pd
from APIkey import APIkey, APIpw
import json
import csv

In [96]:
url= "https://www.rebellegirls.com/yarnfiber.htm"
page= requests.get(url)

soup=BeautifulSoup(page.content, "html.parser")
results=soup.find(id="accordion")
   
#used ChatGPT to figure out this block of webscraping code
stock=[]
name=[]
brands = soup.find_all('h3')
for brand in brands:
    brand_name = brand.text.strip()
    yarn_list = brand.find_next('div').find_all('li')
    for yarn in yarn_list:
        yarn_name = yarn.text.strip()
        name.append(yarn_name)
        stock.append(brand_name)


In [97]:
#create dataframe of Local Yarn Shop (LYS) offerings
stock_list={"stock":stock, "name":name}
rebelle=pd.DataFrame(stock_list)

rebelle['combined']=rebelle['stock']+ '+'+ rebelle['name']
print(rebelle['combined'].iloc[26:51])


26                      Cascade+Ultra Pima
27            Cestari+Mt Vernon Collection
28              Cestari+Old Dominon Cotton
29               Cestari+Traditional 2-ply
30                    Collinette+Jitterbug
31                 Elsbeth Lavold+Hempathy
32            Feederbrook Farms+Entropy DK
33         Feederbrook Farms+Entropy Sport
34                 The Fiber Co.+Arranmore
35      Frabjous Fibers+Recycled Sari Yarn
36             Frabjous Fibers+Silk Ribbon
37       Galler Yarns+Heather Prime Alpaca
38              Hemp for Knitting+Allhemp3
39              Hemp for Knitting+Allhemp6
40               Hemp for Knitting+Hempton
41                    Juniper Moon+Cumulus
42                   Juniper Moon+Zooey DK
43                Kelbourne Woolens+Mojave
44                  Knit Collage+Cast Away
45                 Knit Collage+Pixie Dust
46               Knit Collage+Sampler Kits
47                 Knit Collage+Spun Cloud
48                   Knitted Wit+Fingering
49         

In [98]:
#Code I used to create the ravelry_api.csv file
#Future project is to figure out how to turn this into a FOR LOOP, 
#but couldn't logic it currently and didn't want to waste more time not moving forward on project
#Had to change yarn name in both url and yarn variable, Be sure to CLEAR ALL OUTPUTS after each yarn

#url = 'https://api.ravelry.com/patterns/search.json?yarn=Frabjous+Fibers+Silk+Ribbon'

#response=requests.get(url, auth= (APIkey, APIpw))

#response




In [99]:
#rav_things=response.json()
#patterns= rav_things.get('patterns')
#fiber= pd.DataFrame(response.json()['patterns'])

#fiber['yarn']= 'Frabjous Fibers Silk Ribbon'

#fiber.shape

In [100]:
#fiber.to_csv('ravelry_api.csv', mode='a', index= False, header= False)